# SciPy - Library of scientific algorithms for Python

Parts of this notebook have been taken from:

[http://github.com/jrjohansson/scientific-python-lectures](http://github.com/jrjohansson/scientific-python-lectures).

The other notebooks in this lecture series are indexed at [http://jrjohansson.github.io](http://jrjohansson.github.io).

## Introduction

The SciPy framework builds on top of the low-level NumPy framework for multidimensional arrays, and provides a large number of higher-level scientific algorithms. Some of the topics that SciPy covers are:

* Special functions ([scipy.special](http://docs.scipy.org/doc/scipy/reference/special.html))
* Integration ([scipy.integrate](http://docs.scipy.org/doc/scipy/reference/integrate.html))
* Optimization ([scipy.optimize](http://docs.scipy.org/doc/scipy/reference/optimize.html))
* Interpolation ([scipy.interpolate](http://docs.scipy.org/doc/scipy/reference/interpolate.html))
* Fourier Transforms ([scipy.fftpack](http://docs.scipy.org/doc/scipy/reference/fftpack.html))
* Signal Processing ([scipy.signal](http://docs.scipy.org/doc/scipy/reference/signal.html))
* Linear Algebra ([scipy.linalg](http://docs.scipy.org/doc/scipy/reference/linalg.html))
* Sparse Eigenvalue Problems ([scipy.sparse](http://docs.scipy.org/doc/scipy/reference/sparse.html))
* Statistics ([scipy.stats](http://docs.scipy.org/doc/scipy/reference/stats.html))
* Multi-dimensional image processing ([scipy.ndimage](http://docs.scipy.org/doc/scipy/reference/ndimage.html))
* File IO ([scipy.io](http://docs.scipy.org/doc/scipy/reference/io.html))

Each of these submodules provides a number of functions and classes that can be used to solve problems in their respective topics.

In this lecture we will look at how to use some of these subpackages.

To access the SciPy package in a Python program, we start by importing everything from the `scipy` module.

In [4]:
from scipy import *

If we only need to use part of the SciPy framework we can selectively include only those modules we are interested in. For example, to include the linear algebra package under the name `la`, we can do:

In [6]:
import scipy.linalg as la

### Sparse matrices

Sparse matrices are often useful in numerical simulations dealing with large systems, if the problem can be described in matrix form where the matrices or vectors mostly contains zeros. Scipy has a good support for sparse matrices, with basic linear algebra operations (such as equation solving, eigenvalue calculations, etc).

There are many possible strategies for storing sparse matrices in an efficient way. Some of the most common are the so-called coordinate form (COO), list of list (LIL) form,  and compressed-sparse column CSC (and row, CSR). Each format has some advantanges and disadvantages. Most computational algorithms (equation solving, matrix-matrix multiplication, etc) can be efficiently implemented using CSR or CSC formats, but they are not so intuitive and not so easy to initialize. So often a sparse matrix is initially created in COO or LIL format (where we can efficiently add elements to the sparse matrix data), and then converted to CSC or CSR before used in real calcalations.

For more information about these sparse formats, see e.g. http://en.wikipedia.org/wiki/Sparse_matrix

When we create a sparse matrix we have to choose which format it should be stored in. For example, 

In [7]:
from scipy.sparse import *

In [14]:
# dense matrix
M = array([[1,0,0,0], [0,3,0,0], [0,1,1,0], [1,0,0,1]])
M

array([[1, 0, 0, 0],
       [0, 3, 0, 0],
       [0, 1, 1, 0],
       [1, 0, 0, 1]])

In [15]:
# convert from dense to sparse
A = csr_matrix(M)
A

<4x4 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [16]:
# convert from sparse to dense
A.todense()

matrix([[1, 0, 0, 0],
        [0, 3, 0, 0],
        [0, 1, 1, 0],
        [1, 0, 0, 1]], dtype=int64)

More efficient way to create sparse matrices: create an empty matrix and populate with using matrix indexing (avoids creating a potentially large dense matrix)

In [17]:
A = lil_matrix((4,4)) # empty 4x4 sparse matrix
A[0,0] = 1
A[1,1] = 3
A[2,2] = A[2,1] = 1
A[3,3] = A[3,0] = 1
A

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in LInked List format>

In [18]:
A.todense()

matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  3.,  0.,  0.],
        [ 0.,  1.,  1.,  0.],
        [ 1.,  0.,  0.,  1.]])

### We can compute with sparse matrices like with dense matrices:

In [60]:
(A * A).todense()

matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  9.,  0.,  0.],
        [ 0.,  4.,  1.,  0.],
        [ 2.,  0.,  0.,  1.]])

In [62]:
A.dot(A).todense()

matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  9.,  0.,  0.],
        [ 0.,  4.,  1.,  0.],
        [ 2.,  0.,  0.,  1.]])

In [50]:
A.multiply(A).todense()

matrix([[ 1.,  0.,  0.,  0.],
        [ 0.,  9.,  0.,  0.],
        [ 0.,  1.,  1.,  0.],
        [ 1.,  0.,  0.,  1.]])

## CSR, CSC and COO

#### There are a few different formats in which your sparse matrix might be encoded


#### COO - Coordinate format

In [20]:
A_coo = coo_matrix(A)
A_coo

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in COOrdinate format>

#### We have access to the inner data structure

In [23]:
A_coo.row

array([0, 1, 2, 2, 3, 3], dtype=int32)

In [24]:
A_coo.col

array([0, 1, 1, 2, 0, 3], dtype=int32)

In [25]:
A_coo.data

array([ 1.,  3.,  1.,  1.,  1.,  1.])

### The problem of this data structure is that we cannot efficiently index elements. How can we get the elements in row 3? We would have to loop the row field until we find the first 3, which is in position 5.
### Too slow for matrices of even few hundred rows.

### Wat if we had a pointer to the exact starting position of every row?

### CSR - Compressed Sparse Row format

In [26]:
A_csr = csr_matrix(A)
A_csr

<4x4 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [27]:
A_csr.indices

array([0, 1, 1, 2, 0, 3], dtype=int32)

In [28]:
A_csr.indptr

array([0, 1, 2, 4, 6], dtype=int32)

In [29]:
A_csr.data

array([ 1.,  3.,  1.,  1.,  1.,  1.])

#### In this case
* indices - Refers to the column index, same as .col in COO format
* data - Refers to the value contained in the cell
* indptr - It is a row-pointer. indptr[i] refers to the first cell in indices and data which contains elements of row i

#### Let's extract row 3

In [30]:
target_row = 3

row_start = A_csr.indptr[target_row]
row_end = A_csr.indptr[target_row+1]

In [31]:
row_columns = A_csr.indices[row_start:row_end]
row_columns

array([0, 3], dtype=int32)

In [32]:
row_data = A_csr.data[row_start:row_end]
row_data

array([ 1.,  1.])

#### Double check with the original matrix

In [34]:
M[target_row,:]

array([1, 0, 0, 1])

## WARNING! 
### CSR is made for fast row access, CSC for fast column access. Don't mix them

#### Let's compare the speeds

In [37]:
big_matrix = random(10000, 5000, density=0.05, format='csr')
big_matrix

<10000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 2500000 stored elements in Compressed Sparse Row format>

In [38]:
import time

start_time = time.time()

for row_index in range(big_matrix.shape[0]):
    # Do somenting
    useless_row = big_matrix[row_index]
    
print("Row-wise exploration of a CSR matrix takes {:.2f} seconds".format(time.time()-start_time))

Row-wise exploration of a CSR matrix takes 1.11 seconds


In [39]:
big_matrix = big_matrix.tocsc()

start_time = time.time()

for row_index in range(big_matrix.shape[0]):
    # Do somenting
    useless_row = big_matrix[row_index]
    
print("Row-wise exploration of a CSC matrix takes {:.2f} seconds".format(time.time()-start_time))

Row-wise exploration of a CSC matrix takes 39.47 seconds


### Ouch! There is a 40x difference

#### It's very easy to waste a lot of time by using data structures in an incorrect way

### Other example useful for speed critical code. We want the item indices seen by a user. 
#### The simple implementation would be the following. Users are rows and the items are columns

In [45]:
URM = random(100000, 15000, density=0.01, format='csr')
URM

<100000x15000 sparse matrix of type '<class 'numpy.float64'>'
	with 15000000 stored elements in Compressed Sparse Row format>

In [46]:
start_time = time.time()

for user_id in range(URM.shape[0]):
    # Do somenting
    user_seen_items = URM[user_id].indices
    
print("Extracting user seen items takes {:.2f} seconds".format(time.time()-start_time))

Extracting user seen items takes 6.17 seconds


In [47]:
start_time = time.time()

for user_id in range(URM.shape[0]):
    # Do somenting
    user_seen_items = URM.indices[URM.indptr[user_id]:URM.indptr[user_id+1]]
    
print("Extracting user seen items directly from inner data structure takes {:.2f} seconds".format(time.time()-start_time))

Extracting user seen items directly from inner data structure takes 0.07 seconds


### Even for this simple operation there is a >80x speed difference. Why?

#### Let's see what the first version of the code does...

#### Step 1 - slicing row

In [48]:
user_id = 5
user_row = URM[user_id]
user_row

<1x15000 sparse matrix of type '<class 'numpy.float64'>'
	with 152 stored elements in Compressed Sparse Row format>

#### Step 2 - get column indices

In [49]:
user_seen_items = user_row.indices
user_seen_items

array([  105,   391,   433,   707,  1061,  1169,  1269,  1302,  1512,
        1606,  1667,  1671,  1780,  2183,  2381,  2474,  2523,  2675,
        2682,  2781,  3014,  3244,  3301,  3318,  3646,  3759,  3920,
        3946,  4024,  4122,  4166,  4252,  4278,  4285,  4286,  4326,
        4577,  4586,  4603,  4770,  4785,  4804,  4834,  4839,  4858,
        4871,  4913,  5024,  5041,  5115,  5202,  5297,  5489,  5549,
        5696,  5706,  5761,  5767,  5827,  5855,  5863,  5966,  6035,
        6116,  6216,  6330,  6340,  6359,  6387,  6509,  6581,  6669,
        6854,  6860,  6907,  6918,  6985,  7002,  7018,  7400,  7536,
        8007,  8135,  8209,  8296,  8303,  8573,  8810,  8812,  8891,
        8901,  9094,  9120,  9134,  9356,  9414,  9465,  9487,  9554,
        9562,  9702,  9977,  9988, 10267, 10293, 10333, 10374, 10396,
       10490, 10634, 10687, 10724, 10980, 11013, 11124, 11183, 11334,
       11513, 11622, 11804, 11879, 12142, 12194, 12224, 12296, 12333,
       12407, 12567,

### The reason for the big performance loss is that in this way we are building a new sparse matrix for each user. We don't need the matrix itself, only one of its attributes

## Further reading

* http://www.scipy.org - The official web page for the SciPy project.
* http://docs.scipy.org/doc/scipy/reference/tutorial/index.html - A tutorial on how to get started using SciPy. 
* https://github.com/scipy/scipy/ - The SciPy source code. 